In [1]:
import sys, os
import copy
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print("Added to sys.path:", repo_root)
from fixedincomelib import *
print("Fixed Income Library is loaded.")

Added to sys.path: /Users/lunli/Documents/FixedIncomeLib
Fixed Income Library is loaded.


In [2]:
### utility functions

def bump_reval_interpolator(
    x : float,
    axis1 : List, 
    values : List, 
    interp_method : str,
    extrap_method : str, 
    bump_size : Optional[float]=1e-4):

    base_interpolator = qfCreate1DInterpolator(axis1, values, interp_method, extrap_method)
    b_value = base_interpolator.interpolate(x)

    grad = []
    for i in range(len(values)):
        values[i] += bump_size
        this_interp = qfCreate1DInterpolator(axis1, values, interp_method, extrap_method)
        bumped_value = this_interp.interpolate(x)
        grad.append((bumped_value - b_value) / bump_size)
        values[i] -= bump_size

    return np.array(grad)

def bump_reval_interpolator_integrand(
    x_s : float,
    x_e : float,
    axis1 : List, 
    values : List, 
    interp_method : str,
    extrap_method : str, 
    bump_size : Optional[float]=1e-4):

    base_interpolator = qfCreate1DInterpolator(axis1, values, interp_method, extrap_method)
    b_value = base_interpolator.integrate(x_s, x_e)

    grad = []
    for i in range(len(values)):
        values[i] += bump_size
        this_interp = qfCreate1DInterpolator(axis1, values, interp_method, extrap_method)
        bumped_value = this_interp.integrate(x_s, x_e)
        grad.append((bumped_value - b_value) / bump_size)
        values[i] -= bump_size

    return np.array(grad)


## Test 1D Interpolator -- PCLC

In [3]:
# create a 1d interpolator
axis1 = [1, 3, 5, 7]
values = [3, 4, 5, 6]
interp_method = 'PIECEWISE_CONSTANT_LEFT_CONTINUOUS'
extrap_method = 'FLAT'
interp_1d = qfCreate1DInterpolator(axis1, values, interp_method, extrap_method)

In [4]:
# interpolate
x = 1.5
print(f'At {x}, the interpolated value is {interp_1d.interpolate(x)}.')
# extrpolate left
x = 0.5
print(f'At {x}, the interpolated value is {interp_1d.interpolate(x)}.')
# extrpolate right
x = 4.5
print(f'At {x}, the interpolated value is {interp_1d.interpolate(x)}.')

At 1.5, the interpolated value is 3.
At 0.5, the interpolated value is 3.
At 4.5, the interpolated value is 4.


In [5]:
### interpolator sensitivity
for x in [1.5, 0.5, 4.5]:
    # analytic
    grad_analytic = interp_1d.gradient_wrt_ordinate(x)
    # bump reval
    grad_br = bump_reval_interpolator(x, axis1, values, interp_method, extrap_method)
    # assertion
    print(f'With {x}, the diff is {(grad_analytic - grad_br).sum()}.')

With 1.5, the diff is -2.1103119252074976e-12.
With 0.5, the diff is -2.1103119252074976e-12.
With 4.5, the diff is -2.1103119252074976e-12.


In [6]:
# integrand
# 1) both out of left wing
x_s, x_e = 0.5, 0.9
print(f'From {x_s} to {x_e}, the integrand value is {interp_1d.integrate(x_s, x_e)}.')
# 2) one left wing, one in the first bucket
x_s, x_e = 0.5, 1.2
print(f'From {x_s} to {x_e}, the integrand value is {interp_1d.integrate(x_s, x_e)}.')
# 3) one left wing, one in the middle bucket
x_s, x_e = 0.5, 3.2
print(f'From {x_s} to {x_e}, the integrand value is {interp_1d.integrate(x_s, x_e)}.')
# 4) both in the middle
x_s, x_e = 1.5, 5.2
print(f'From {x_s} to {x_e}, the integrand value is {interp_1d.integrate(x_s, x_e)}.')
# 5) one in middle bucket, one on the right wing
x_s, x_e = 3.5, 7.2
print(f'From {x_s} to {x_e}, the integrand value is {interp_1d.integrate(x_s, x_e)}.')
# 6) one in the last bucket, one on the right wing
x_s, x_e = 6, 7.2
print(f'From {x_s} to {x_e}, the integrand value is {interp_1d.integrate(x_s, x_e)}.')
# 7) both right wing
x_s, x_e = 8, 10
print(f'From {x_s} to {x_e}, the integrand value is {interp_1d.integrate(x_s, x_e)}.')
# 8) complete accross
x_s, x_e = 0.1, 10
print(f'From {x_s} to {x_e}, the integrand value is {interp_1d.integrate(x_s, x_e)}.')

From 0.5 to 0.9, the integrand value is 1.2000000000000002.
From 0.5 to 1.2, the integrand value is 2.0999999999999996.
From 0.5 to 3.2, the integrand value is 8.3.
From 1.5 to 5.2, the integrand value is 13.5.
From 3.5 to 7.2, the integrand value is 17.200000000000003.
From 6 to 7.2, the integrand value is 6.200000000000001.
From 8 to 10, the integrand value is 12.0.
From 0.1 to 10, the integrand value is 44.7.


In [7]:
### interpolator integral sensitivity
test_cases = [
    [0.5, 0.9],
    [0.5, 1.2],
    [0.5, 3.2],
    [1.5, 5.2],
    [3.5, 7.2],
    [6, 7.2],
    [8, 10],
    [0.1, 10]
]
for x_s, x_e in test_cases:
    grad_analytic = interp_1d.gradient_of_integrated_value_wrt_ordinate(x_s, x_e)
    grad_br = bump_reval_interpolator_integrand(x_s, x_e, axis1, values, interp_method, extrap_method)
    # assertion
    print(f'With {x_s} and {x_e}, the diff is {(grad_analytic - grad_br).sum()}.')

With 0.5 and 0.9, the diff is -4.000133557724439e-13.
With 0.5 and 1.2, the diff is -4.585887225516672e-12.
With 0.5 and 3.2, the diff is 1.3398171461176389e-11.
With 1.5 and 5.2, the diff is -1.979838515353549e-11.
With 3.5 and 7.2, the diff is 3.349232002847202e-11.
With 6 and 7.2, the diff is 1.0205170042354439e-12.
With 8 and 10, the diff is 4.661160346586257e-12.
With 0.1 and 10, the diff is 2.6820989873499457e-10.
